In [9]:
import mysql.connector
import psycopg2
import pandas as pd
import os
import json

# Carregar configurações do arquivo
with open('config.json') as f:
    config = json.load(f)

# Configurações de conexão com o banco de dados MySQL
db_config = config['mysql']

# Configurações de conexão com o banco de dados Postgres
db_config_postgres = config['postgres']


# STAGE ALMOCO
def load_stg(df):
    try:
        # Conectar ao banco de dados
        conn = psycopg2.connect(**db_config_postgres)
        
        # Criar um cursor
        cursor = conn.cursor()
        
        # Iniciar a transação
        cursor.execute("BEGIN;")

        
        # Criar a tabela de fatos se ela ainda não existir
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS "STG".stg_almoco (  -- Mantenha "STG" dentro das aspas duplas
            nome varchar(100) NULL,
            tipo_usuario varchar(17) NULL,
            empresa varchar(100) NULL,
            hora timestamp NULL,
            tipo_refeicao varchar(13) NULL,
            turno varchar(11) NULL
        )
        """)
        
        # Inserir os dados na tabela de fatos
        for _, row in df.iterrows():
            cursor.execute('''
                INSERT INTO "STG".stg_almoco (  -- Mantenha "STG" dentro das aspas duplas
                    nome,
                    tipo_usuario,
                    empresa,
                    hora,
                    tipo_refeicao,
                    turno
                )
                VALUES (
                    %s, %s, %s, %s, %s, %s
                )
            ''', (
                row['NOME'],
                row['TIPO_USUARIO'],
                row['EMPRESA'],
                row['HORA'],
                row['TIPO_REFEICAO'],
                row['TURNO'],

            ))
        
        # Commit a transação
        cursor.execute("COMMIT;")
        
        print("Dados carregados na tabela de STG.almoco com sucesso.")
    except Exception as e:
            # Rollback se ocorrer algum erro
            cursor.execute("ROLLBACK;")
            print("Ocorreu um erro:", e)
    finally:
            # Fechar conexão e cursor
            cursor.close()
            conn.close()



# STAGE ALMOCO
def load_DW(df):
    try:
        # Conectar ao banco de dados
        conn = psycopg2.connect(**db_config_postgres)
        
        # Criar um cursor
        cursor = conn.cursor()
        
        # Iniciar a transação
        cursor.execute("BEGIN;")

        
        # Criar a tabela de fatos se ela ainda não existir
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS "DW".f_almoco (  
            nome varchar(100) NULL,
            tipo_usuario varchar(17) NULL,
            empresa varchar(100) NULL,
            hora timestamp NULL,
            tipo_refeicao varchar(13) NULL,
            turno varchar(11) NULL
        )
        """)
        
        # Inserir os dados na tabela de fatos
        for _, row in df.iterrows():
            cursor.execute('''
                INSERT INTO "DW".f_almoco_teste ( 
                    nome,
                    tipo_usuario,
                    empresa,
                    hora,
                    tipo_refeicao,
                    turno
                )
                VALUES (
                    %s, %s, %s, %s, %s, %s
                )
            ''', (
                row['nome'],
                row['tipo_usuario'],
                row['empresa'],
                row['hora'],
                row['tipo_refeicao'],
                row['turno'],

            ))
        
        # Commit a transação
        cursor.execute("COMMIT;")
        
        print("Dados carregados na tabela de DW.f_almoco com sucesso.")
    except Exception as e:
            # Rollback se ocorrer algum erro
            cursor.execute("ROLLBACK;")
            print("Ocorreu um erro:", e)
    finally:
            # Fechar conexão e cursor
            cursor.close()
            conn.close()


# Função para executar uma consulta SQL e retornar os resultados como um DataFrame do Pandas
def query_to_dataframe(query):
    try:
        # Conectar ao banco de dados Postgres
        conn = mysql.connector.connect(**db_config)
        
        # Criar um cursor
        cursor = conn.cursor()
        
        # Executar a consulta SQL
        cursor.execute(query)
        
        # Obter os resultados da consulta
        data = cursor.fetchall()
        
        # Obter os nomes das colunas
        columns = [desc[0] for desc in cursor.description]
        
        # Criar um DataFrame do Pandas com os resultados
        df = pd.DataFrame(data, columns=columns)
        
        # Fechar o cursor e a conexão
        cursor.close()
        conn.close()
        
        return df
    except Exception as e:
        print("Ocorreu um erro:", e)
        return None


def query_postgres_to_dataframe(query):
    try:
        # Conectar ao banco de dados Postgres
        conn = psycopg2.connect(**db_config_postgres)
        
        # Criar um cursor
        cursor = conn.cursor()
        
        # Executar a consulta SQL
        cursor.execute(query)
        
        # Obter os resultados da consulta
        data = cursor.fetchall()
        
        # Obter os nomes das colunas
        columns = [desc[0] for desc in cursor.description]
        
        # Criar um DataFrame do Pandas com os resultados
        df = pd.DataFrame(data, columns=columns)
        
        # Fechar o cursor e a conexão
        cursor.close()
        conn.close()
        
        return df
    except Exception as e:
        print("Ocorreu um erro:", e)
        return None



data_inicio = '2024/02/20 00:00:00.000'
data_fim = '2024/03/11 23:59:59.000'


query_stg_almoco = f""" 
SELECT *,case
	    WHEN TIME(HORA) BETWEEN '10:00:00' AND '13:30:00' THEN 'ALMOCO'
        WHEN TIME(HORA) BETWEEN '06:00:00' AND '08:00:00' THEN 'CAFE DA MANHA'
        WHEN TIME(HORA) BETWEEN '14:00:00' AND '17:00:00' THEN 'CAFE DA TARDE'
        WHEN TIME(HORA) BETWEEN '20:00:00' AND '23:00:00' THEN 'JANTAR'
    END AS TIPO_REFEICAO,
    case
	    -- Cafe da manha
	    WHEN TIME(HORA) BETWEEN '06:00:00' AND '06:30:00' THEN '06:00-06:30'
        WHEN TIME(HORA) BETWEEN '06:30:00' AND '07:00:00' THEN '06:30-07:00'
	    WHEN TIME(HORA) BETWEEN '07:00:00' AND '07:30:00' THEN '07:00-07:30'
        WHEN TIME(HORA) BETWEEN '07:30:00' AND '08:00:00' THEN '07:30-08:00'
        
        -- Cafe da tarde
	    WHEN TIME(HORA) BETWEEN '14:00:00' AND '15:00:00' THEN '14:00-15:00'
        WHEN TIME(HORA) BETWEEN '15:00:00' AND '16:00:00' THEN '15:00-16:00'
	    WHEN TIME(HORA) BETWEEN '16:00:00' AND '17:00:00' THEN '16:00-17:00'

	    -- Almoco
	    WHEN TIME(HORA) BETWEEN '10:00:00' AND '10:30:00' THEN '10:00-10:30'
        WHEN TIME(HORA) BETWEEN '10:30:00' AND '11:00:00' THEN '10:30-11:00'
        WHEN TIME(HORA) BETWEEN '11:00:01' AND '11:30:00' THEN '11:00-11:30'
        WHEN TIME(HORA) BETWEEN '11:30:01' AND '12:00:00' THEN '11:30-12:00'
        WHEN TIME(HORA) BETWEEN '12:00:01' AND '12:30:00' THEN '12:00-12:30'
        WHEN TIME(HORA) BETWEEN '12:30:01' AND '13:00:00' THEN '12:30-13:00'
        WHEN TIME(HORA) BETWEEN '13:00:01' AND '13:30:00' THEN '13:00-13:30'

        -- Jantar
	    WHEN TIME(HORA) BETWEEN '20:00:00' AND '21:00:00' THEN '20:00-21:00'
        WHEN TIME(HORA) BETWEEN '21:00:00' AND '22:00:00' THEN '21:00-22:00'
	    WHEN TIME(HORA) BETWEEN '22:00:00' AND '23:00:00' THEN '22:00-23:00'
	    
        END AS TURNO
FROM (
    SELECT logs.TIME AS HORA
        ,UPPER(users.name) AS NOME
        ,CASE 
            WHEN users.id IN (
                    SELECT idUser
                    FROM usergroups
                    WHERE idGroup IN (
                            1024
                            ,1025
                            ,1026
                            ,1027
                            ,1029
                            ,1030
                            ,1031
                            )
                    )
                THEN 'Visitante'
            WHEN users.id = 1002
                THEN 'Terceiro Fixo'
            WHEN users.id = 1003
                THEN 'Terceiro Eventual'
            ELSE 'Funcionário'
            END AS TIPO_USUARIO
        ,CASE 
            WHEN users.visitorCompany IS NOT NULL
                THEN UPPER(users.visitorCompany)
            ELSE 'Grupo Tristão'
            END AS EMPRESA
    FROM logs
    INNER JOIN (
        SELECT idUser
            ,DATE(TIME) AS data
            ,MAX(CASE 
                    WHEN TIME BETWEEN CONCAT(DATE(TIME), ' 10:00:00') AND CONCAT(DATE(TIME), ' 13:30:00') THEN TIME -- Almoco
                    WHEN TIME BETWEEN CONCAT(DATE(TIME), ' 06:00:00') AND CONCAT(DATE(TIME), ' 08:00:00') THEN TIME -- Café da manhã
                    WHEN TIME BETWEEN CONCAT(DATE(TIME), ' 14:00:00') AND CONCAT(DATE(TIME), ' 17:00:00') THEN TIME -- Café da Tarde
                    WHEN TIME BETWEEN CONCAT(DATE(TIME), ' 20:00:00') AND CONCAT(DATE(TIME), ' 23:00:00') THEN TIME -- Jantar
                    END) AS HORA
        FROM logs
        WHERE idDevice = 44
            AND event != 3
            
			  AND DATE(TIME) BETWEEN '{data_inicio}' AND '{data_fim}' -- Intervalo de uma semana
			
			
		GROUP BY idUser, DATE(TIME) -- Agrupar por usuário e data
        ) AS latest_logs ON logs.idUser = latest_logs.idUser
        AND logs.TIME = latest_logs.HORA
        AND DATE(logs.TIME) = latest_logs.data
    INNER JOIN users ON logs.idUser = users.id
    ) T1
ORDER BY T1.HORA;

 """


query_f_almoco = """ SELECT * FROM "DW".f_almoco """

data_stg_almoco = query_to_dataframe(query_stg_almoco)

load_stg(data_stg_almoco)

data_f_almoco = query_postgres_to_dataframe(query_f_almoco)

data_f_almoco.head()

load_DW(data_f_almoco)






Dados carregados na tabela de STG.almoco com sucesso.
Dados carregados na tabela de DW.f_almoco_teste com sucesso.
